# Crack processing

The crack propagation and state characterization is done using the class `DICCrack` which represents its geometry in form of a spline.
Cracks are contained in the `DICCrackList` and can be accessed using the index operator with the crack label. Currently, the crack label is an enumeration running from left to right. 

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrackList, \
    DICAlignedGrid
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
import collections
collections.Iterable = collections.abc.Iterable

In [ ]:
dic_grid = DICGrid(U_factor=100, dir_name='B7_TV1', t=1)

In [ ]:
dic_grid.read_beam_design()

In [ ]:
from bmcs_shear.dic_crack import DICStressProfile
dsf = DICStateFields(dic_grid=dic_grid, R=8, omega_threshold=0.05)

In [ ]:
l_cb = dic_grid.d_x * 1
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.02/l_cb, eps_max=0.005);
dsf.eval()

In [ ]:
dcl = DICCrackList(dsf=dsf, corridor_left=3, corridor_right=0)
dcl.identify_cracks()

In [ ]:
dc =dcl.items['0']

In [ ]:
dc.x_1_tip_a, dc.y_N, 

In [ ]:
self = dc
_, y_tip = self.x_1_tip_a - self.y_N[0]
# Cracked fraction of cross-section
d_y = y_tip / self.H_ligament
y_tip

In [ ]:
self = dc
# array of tip indexes for each dic time step
K_tip_T = np.zeros((self.dic_grid.n_T,), dtype=np.int_)
# consider only steps with at least one non-zero damage value
K_omega_T = np.where(np.sum(self.omega_TK, axis=-1) > 0)[0]
# search from the top of the ligament the first occurrence of damage
# larger than threshold. The search starts from the crack tip identified
# for the ultimate state and goes downwards to the point where the
# damage reaches the overall damage threshold omega_threshold.
L_tip_1 = self.n_K_ligament - self.K_tip_1
# for each time step get the first indexes with omega larger than threshold
L_omega = np.argmax(np.flip(self.omega_TK[K_omega_T], axis=-1)[:, L_tip_1:] >
                    self.omega_threshold, axis=-1)
# identify the indexes of the current crack tip from the bottom of the ligament
K_omega_tip_T = self.K_tip_1 - L_omega
# place the found indexes into the time array
K_tip_T[K_omega_T] = K_omega_tip_T

In [ ]:
K_tip_T

In [ ]:
dc.interact()

In [ ]:
dc.x_1_tip_a

In [ ]:
dc.x_t_tip_a

In [ ]:
dc.T_t

In [ ]:
dc.dic_grid.T_t

In [ ]:
dc.K_tip_T[-1], dc.K_tip_1/dc.n_K_ligament

In [ ]:
dc.N_tip / len(dc.M_N)

In [ ]:
dc.x_1_Ka[96,1]

In [ ]:
dc.y_N[dc.N_tip]

In [ ]:
dc.x_1_Ka[99,1], dc.y_N[dc.N_tip]

In [ ]:
np.argmax(dc.x_1_Ka[:,1] > dc.x_1_tip_a[1]) - 1